# CIFAR-10 Dataset

## Description

The CIFAR-10 dataset contains colour images that can be classified in 10 classes:

- airplane										
- automobile										
- bird										
- cat										
- deer										
- dog										
- frog										
- horse										
- ship										
- truck

## Importing Dataset

In [ ]:
# Manage Imports
import pandas as pd
import numpy as np
import sklearn.preprocessing

# Manage global variables
NUMBER_OF_FILES = 5
IMAGES_PER_FILE = 10000
IMAGE_SIZE = 32
NUMBER_OF_CHANNELS = 3

In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
metadata = unpickle(f'../../datasets/image/cifar-10-batches-py/batches.meta')
for key, value in metadata.items():
    print(f'{key=}, {value}')

In [ ]:
X_train = np.zeros((NUMBER_OF_FILES * IMAGES_PER_FILE, IMAGE_SIZE*IMAGE_SIZE*NUMBER_OF_CHANNELS), dtype=np.uint8)
y_train = np.zeros((NUMBER_OF_FILES*IMAGES_PER_FILE), dtype=np.int32)

for i in range(5):
    batch = unpickle(f'../../datasets/image/cifar-10-batches-py/data_batch_{i+1}')
    X_train[i*10000:(i+1)*10000] = np.array(batch[b'data'])
    y_train[i*10000:(i+1)*10000] = np.array(batch[b'labels'])

In [ ]:
from sklearn.model_selection import train_test_split

test_batch = unpickle(f'../../datasets/image/cifar-10-batches-py/test_batch')
X_test_and_val = np.array(batch[b'data'])
y_test_and_val = np.array(batch[b'labels'])
X_test, X_val, y_test, y_val = train_test_split(X_test_and_val, y_test_and_val, 
                                                        train_size=0.5,
                                                        random_state=0)


The following block prints the shape and column datatypes of the image dataset.

In [ ]:
print(f'{X_train.shape=}, {X_train.dtype=}')
print(f'{y_train.shape=}, {y_train.dtype=}')
print(f'{X_test.shape=}, {X_test.dtype=}')
print(f'{y_test.shape=}, {y_test.dtype=}')
print(f'{X_val.shape=}, {X_val.dtype=}')
print(f'{y_val.shape=}, {y_val.dtype=}')

The following block shows how to preview an image. It is important to note that a single CIFAR image is stored in (N,C,H,W) format.

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(X_train[0].reshape(IMAGE_SIZE, IMAGE_SIZE, NUMBER_OF_CHANNELS))
plt.figure()
plt.imshow(X_train[0].reshape(NUMBER_OF_CHANNELS, IMAGE_SIZE, IMAGE_SIZE).transpose(1, 2, 0))

## Training on Multiple Classifiers

In [ ]:
# Manage imports
import sklearn.tree
import torch
import torchvision
from utilities import train_estimators, plot_estimator_scores

### Decision Tree Classification

In [9]:
adjusted_parameter = 'max_depth'
adjusted_parameter_values = [1, 5, 10, 20, 50, 100]

DecisionTreeEstimators = train_estimators(X_train, y_train,
                                            sklearn.tree.DecisionTreeClassifier,
                                            adjusted_parameter, adjusted_parameter_values,
                                            splitter='random',
                                            random_state=0)
plot_estimator_scores(DecisionTreeEstimators,
                        adjusted_parameter, adjusted_parameter_values,
                        X_train, y_train, X_test, y_test, X_val, y_val)

### Neural Network Classification Through a Convolutional Neural Network

In [ ]:
X_train_torch = torch.tensor(X_train, dtype=torch.uint8)
y_train_torch = torch.tensor(y_train, dtype=torch.int32)
X_test_torch = torch.tensor(X_test, dtype=torch.uint8)
y_test_torch = torch.tensor(y_test, dtype=torch.int32)
X_val_torch = torch.tensor(X_val, dtype=torch.uint8)
y_val_torch = torch.tensor(y_val, dtype=torch.int32)

print(X_train_torch.dtype)
print(y_train_torch.dtype)
print(X_test_torch.dtype)
print(y_test_torch.dtype)
print(X_val_torch.dtype)
print(y_val_torch.dtype)

In [ ]:
torch.manual_seed(0) # Ensure model weights initialized with same random numbers

# Use 100 training samples at a time to compute the gradient.
batch_size = 1

# Create an object that holds a sequence of layers and activation functions
model = torch.nn.Sequential(
    torch.nn.Conv2d(NUMBER_OF_CHANNELS, batch_size, IMAGE_SIZE, IMAGE_SIZE),
)

# Create an object that can compute "negative log likelihood of a softmax"
loss = torch.nn.CrossEntropyLoss()
# Use stochastic gradient descent to train the model
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Make 10 passes over the training data, each time using batch_size samples to compute
num_epoch = 10
next_epoch = 1

for epoch in range(next_epoch, next_epoch+num_epoch):
    
    # Make an entire pass (an 'epoch') over the training data in batch_size chunks
    for i in range(0, len(X_train_torch), batch_size):        
        X = X_train_torch[i:i+batch_size].reshape((batch_size, NUMBER_OF_CHANNELS, IMAGE_SIZE, IMAGE_SIZE))  # Slice out a mini-batch of features
        y = y_train_torch[i:i+batch_size]     # Slice out a mini-batch of targets

        y_pred = model(X)                   # Make predictions (final-layer activations)
        l = loss(y_pred, y_train_torch)     # Compute loss with respect to predictions
        
        model.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
        l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
        optimizer.step()                    # Use the gradients to take a step with SGD.
        
    print("Epoch %2d: loss on final training batch: %.4f" % (epoch, l.item()))
    
print("Epoch %2d: loss on test set: %.4f" % (epoch, loss(model(X_test_torch), y_test_torch)))
next_epoch = epoch+1